# etl: Transform

In [104]:
import polars as pl
import re 
import time
def view_string(long_string, chunk_size=100):     
    return [long_string[i:i+chunk_size] for i in range(0, len(long_string), chunk_size)]

In [105]:
df_clean = pl.read_csv(   
    '/home/sebacastillo/willow/output/news_narcotráfico_related_2023-08-16_1355.csv',
    dtypes={"content_hash": pl.UInt64},     
)

In [106]:
print(df_clean.shape), print(df_clean.columns)

(44, 14)
['date_extract', 'date_article', 'topic', 'content', 'link', 'link_sim_score', 'title', 'summary', 'authors', 'portal', 'state', 'city', 'content_hash', 'content_nchar']


(None, None)

In [107]:
df_clean.head(1)

date_extract,date_article,topic,content,link,link_sim_score,title,summary,authors,portal,state,city,content_hash,content_nchar
str,str,str,str,str,f64,str,str,str,str,str,str,u64,i64
"""2023-08-16 13:…","""2023-08-16T13:…","""narcotráfico""","""Seguinos Por L…","""https://www.la…",0.561929,"""Condenaron a s…","""Brian está det…","""n-a""","""https://www.ar…","""Santa Fe""","""Rosario""",9259959111029284149,4543


In [20]:
df['content'].to_list()[3]

' El usuario o la contraseña son incorrectos. Si ya estás registrado en Diario el Tribuno de Salta\n\t\t\t                    \t\t            completá tu usuario y contraseña: Error durante el registro.   \n14 DE Agosto 2023 - 03:48\n                                Especialistas salteños desmitificaron algunos secretos.\n                             El pasado 26 de julio, un comunicado de la Administración Nacional de Medicamentos, Alimentos y Tecnología Médica (ANMAT) generó revuelo. La autorización de inscripción de la ya famosa Ozempic, la droga de moda que usan los famosos para bajar de peso, hizo pensar a muchos que su milagro estaba en puertas. En algunos países de Europa y en Estados Unidos, este medicamento ganó su fama en las redes sociales, en donde se muestra que el Ozempic sirve para perder peso. Incluso hay celebridades de la talla de Lady Gaga, Kim Kardashian y hasta Elon Musk que reconocen haberlo consumido, sin mediar en consecuencias. El fármaco es un medicamento inyec

# Cleaning febore write to DB

In [83]:
df_clean = (
    df_clean.with_columns([
        pl.col('date_extract').str.strptime(pl.Date, format='%Y-%m-%d %H:%M:%s', strict=True),
        pl.col('date_article').str.slice(0, 10).str.strptime(pl.Date, format='%Y-%m-%d'),        
        
        # Replace new lines and tabs with a single space, then replace multiple spaces with a single space
        pl.col("content").str.replace_all(r"[\n\t]+", " ")
        .str.replace_all(r"\s{2,}", " ")
        .str.strip().str.strip().alias("content"),  # Adjusted
    ])
)


In [84]:
df_clean.head()

date_extract,date_article,topic,content,link,link_sim_score,title,summary,authors,portal,state,city,content_hash,content_nchar,link_hash
date,date,str,str,str,f64,str,str,str,str,str,str,u64,i64,u64
2023-08-16,2023-08-16,"""narcotráfico""","""Seguinos Por L…","""https://www.la…",0.561929,"""Condenaron a s…","""Brian está det…","""n-a""","""https://www.ar…","""Santa Fe""","""Rosario""",9259959111029284149,4543,17052477334313902255
2023-08-16,2023-08-16,"""narcotráfico""","""Escuadrón 1 “R…","""http://www.dia…",0.476911,"""Encuentran 32 …","""na""","""Diario El Oran…","""https://www.ar…","""Salta""","""Orán""",6046667906668190206,707,3209048333592772804
2023-08-16,2023-08-16,"""narcotráfico""",""". Compartir en…","""https://www.ar…",0.526558,"""Prefectura sec…","""Efectivos de l…","""n-a""","""https://www.ar…","""Argentina""","""Argentina""",12244461015022324118,657,16967809217027424855
2023-08-16,2023-08-14,"""narcotráfico""","""El usuario o l…","""https://www.el…",0.540131,"""Semaglutida: l…","""También ""fidel…","""El Tribuno""","""https://www.ar…","""Jujuy""","""San Salvador d…",13466443961547567731,23475,14376176470560487263
2023-08-16,2023-08-16,"""narcotráfico""","""Escuadrón 54 “…","""http://www.dia…",0.503642,"""Secuestran más…","""na""","""Diario El Oran…","""https://www.ar…","""Salta""","""Orán""",8279679033715743567,1293,2411650459581365560


# Agrego summary2_llm

In [6]:
articles = df_clean['content'].to_list()

for article in articles[:3]:
    print(article)

Seguinos Por Leo Graciarena La jueza Elena Beatriz Dilario, miembro del Tribunal Oral Federal 3, homologó el abreviado. Seis personas sindicadas como integrantes del “Clan Insaurralde” fueron condenadas por el Tribunal Oral Federal 3 en un proceso abreviado como responsables, en diferentes grados, del delito de tráfico y tenencia de estupefacientes con fines de comercialización agravada por la intervención de tres o más personas en forma organizada. La jueza Elena Beatriz Dilario homologó el acuerdo al que llegaron las partes. Entre los condenados se encuentra Brian Nahuel Insaurralde, quien es sobrino del jefe del clan, Ramón Javier “Rengo” Insaurralde (quien se encuentra cumpliendo sentencia), y que recibió una pena unificada de 11 años y 6 meses; y una de las proveedoras del grupo, Roxana García Sipe, a quien la dieron una pena única de 8 años y 6 meses de cárcel. Brian está detenido en la cárcel de Villa Devoto, en la Ciudad Autónoma de Buenos Aires, mientras que García Sipe está b

In [36]:
view_string(articles[0])

['Seguinos Por Leo Graciarena La jueza Elena Beatriz Dilario, miembro del Tribunal Oral Federal 3, hom',
 'ologó el abreviado. Seis personas sindicadas como integrantes del “Clan Insaurralde” fueron condenad',
 'as por el Tribunal Oral Federal 3 en un proceso abreviado como responsables, en diferentes grados, d',
 'el delito de tráfico y tenencia de estupefacientes con fines de comercialización agravada por la int',
 'ervención de tres o más personas en forma organizada. La jueza Elena Beatriz Dilario homologó el acu',
 'erdo al que llegaron las partes. Entre los condenados se encuentra Brian Nahuel Insaurralde, quien e',
 's sobrino del jefe del clan, Ramón Javier “Rengo” Insaurralde (quien se encuentra cumpliendo sentenc',
 'ia), y que recibió una pena unificada de 11 años y 6 meses; y una de las proveedoras del grupo, Roxa',
 'na García Sipe, a quien la dieron una pena única de 8 años y 6 meses de cárcel. Brian está detenido ',
 'en la cárcel de Villa Devoto, en la Ciudad Autónoma d

In [ ]:
#embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
    #                                   model_kwargs={'device': 'cpu'})
#embeddings = HuggingFaceEmbeddings(model_name='thenlper/gte-base',
#                                       model_kwargs={'device': 'cpu'})

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_str = "IIC/mt5-spanish-mlsum"
#model_Str = "dccuchile/bert-base-spanish-wwm-cased"
tokenizer = AutoTokenizer.from_pretrained(model_str)
model = AutoModelForSeq2SeqLM.from_pretrained(model_str)

In [9]:
articles_summaries = []
for article in articles:
    input_ids = tokenizer(article, return_tensors="pt").input_ids    
    output_ids = model.generate(input_ids, max_new_tokens=400, penalty_alpha=0.6, top_k=4,)[0]
    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    articles_summaries.append(summary)

In [29]:
view_string(articles_summaries[5])

['Fuerza Naval Argentina acata marihuana y armas blancas en Villa La Angostura. La Fuerza Naval Argent',
 'ina incautó a tres personas, dos mujeres y un hombre']

In [14]:
print(len(articles_summaries)), print(df_clean.shape[0])

119
119


(None, None)

In [11]:
summaries = pl.DataFrame(articles_summaries)
df_clean = df_clean.with_columns(pl.Series("summary_llm", articles_summaries))  


In [12]:
df_clean.columns

['date_extract',
 'date_article',
 'topic',
 'content',
 'link',
 'link_sim_score',
 'title',
 'summary',
 'authors',
 'portal',
 'state',
 'city',
 'content_hash',
 'content_nchar',
 'summary_llm']

In [13]:
df_clean.write_csv('df_clean.csv')

In [14]:
del tokenizer
del model

In [15]:
import gc
gc.collect()

0

# Agrego NER 

In [30]:
index = list(range(1, df_clean.shape[0]+1))
articles = df_clean['content'].to_list()

In [31]:
import spacy
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline

# Load Spanish stopwords from spacy
#nlp = spacy.load("es_core_news_sm")
#spanish_stopwords_spacy = spacy.lang.es.stop_words.STOP_WORDS

#def remove_stopwords(text):
#    """Remove Spanish stopwords from the text."""
#    return " ".join([word for word in text.split() if word not in spanish_stopwords_spacy])

def langchain_chunk_text(text):
    
    # Remove stopwords
    # text = remove_stopwords(text)
    
    # Define a basic whitespace tokenizer
    def whitespace_tokenizer(text):
        return text.split()

    custom_text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=60,
        length_function = len,
        #length_function=lambda x: len(whitespace_tokenizer(x))
    )
    
    # Create the chunks using langchain
    documents = custom_text_splitter.create_documents([text])
    
    # Extract the text content from the resulting Document objects
    chunks = [doc.page_content for doc in documents]
    
    return chunks


def ner_on_large_document(text):
    nlp_ner = pipeline(
        "ner",
        model="mrm8488/bert-spanish-cased-finetuned-ner",
        #model="PlanTL-GOB-ES/roberta-base-bne-capitel-ner",
        #tokenizer=('mrm8488/bert-spanish-cased-finetuned-ner', {"use_fast": False}),
        #aggregation_strategy="simple"
        aggregation_strategy="max" # more precision!
        #aggregation_strategy="average"
    )

    chunks = langchain_chunk_text(text)
    all_ner_results = []

    for chunk in chunks:
        ner_results = nlp_ner(chunk)
        all_ner_results.extend(ner_results)

    return all_ner_results

In [34]:
from tqdm import tqdm

In [ ]:
# Initialize an empty DataFrame
df_combined = pl.DataFrame()
for article, i in tqdm(zip(articles, index)):
    # Assume ner_on_large_document is a function that processes the article
    ner = ner_on_large_document(article)
    df = pl.DataFrame(ner)
    df_len = df.shape[0]
    df = df.with_columns(pl.Series("index", [i] * df_len))  
    # Check if the columns of df match those of df_combined or if df_combined is still empty
    if set(df.columns) == set(df_combined.columns) or df_combined.shape[1] == 0:
        # Vertically stack the generated df to the df_combined
        df_combined = pl.concat([df_combined, df])

In [95]:
#df_combined.write_csv('ner.csv')
df_combined = pl.read_csv('ner.csv')

In [109]:
df_clean = df_clean.with_columns(pl.Series("index", index))  

In [100]:
# Perform the join operation directly using the 'index' column as the key
df_combined = df_combined.join(df_clean[['link', 'content_hash', 'index']], on='index', how='left')


In [101]:
df_combined

entity_group,score,word,start,end,index,link,content_hash
str,f64,str,i64,i64,i64,str,u64
"""PER""",0.9998,"""Leo Graciarena…",13,27,1,"""https://www.la…",9259959111029284149
"""PER""",0.999691,"""Elena Beatriz …",37,58,1,"""https://www.la…",9259959111029284149
"""ORG""",0.984321,"""Tribunal Oral …",72,95,1,"""https://www.la…",9259959111029284149
"""ORG""",0.98353,"""“ Clan Insaurr…",166,184,1,"""https://www.la…",9259959111029284149
"""ORG""",0.997706,"""Tribunal Oral …",210,233,1,"""https://www.la…",9259959111029284149
"""PER""",0.999829,"""Elena Beatriz …",112,133,1,"""https://www.la…",9259959111029284149
"""PER""",0.99977,"""Brian Nahuel I…",216,240,1,"""https://www.la…",9259959111029284149
"""PER""",0.999559,"""Ramón Javier""",278,290,1,"""https://www.la…",9259959111029284149
"""PER""",0.993575,"""“ Rengo ” Insa…",291,310,1,"""https://www.la…",9259959111029284149


In [110]:
df_clean.columns

['date_extract',
 'date_article',
 'topic',
 'content',
 'link',
 'link_sim_score',
 'title',
 'summary',
 'authors',
 'portal',
 'state',
 'city',
 'content_hash',
 'content_nchar',
 'index']

In [ ]:
df_reordered = df_combined.select([
    'index',
    'topic',
    'date_extract',
    'date_article',    
    'content',
    'portal',
    'link',
    'link_sim_score',
    'title',
    'summary',
    'summary_llm',
    'authors',    
    'state',
    'city',
    'content_hash',
    'content_nchar'
])

In [111]:
df_combined.columns

['entity_group',
 'score',
 'word',
 'start',
 'end',
 'index',
 'link',
 'content_hash']

In [ ]:
df_combined = df_combined.select(
    [
        'index',
        'link',
        'content_hash'
        'entity_group',
        'score',
        'word',
        'start',
        'end'        
    ]
)

In [ ]:
import spacy
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline


def langchain_chunk_text(text):
   
    custom_text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=60,
        length_function = len        
    )
    
    # Create the chunks using langchain
    documents = custom_text_splitter.create_documents([text])
    
    # Extract the text content from the resulting Document objects
    chunks = [doc.page_content for doc in documents]
    
    return chunks


def ner_on_large_document(text):
    nlp_ner = pipeline(
        "ner",
        model="mrm8488/bert-spanish-cased-finetuned-ner",
        #model="PlanTL-GOB-ES/roberta-base-bne-capitel-ner",
        #tokenizer=('mrm8488/bert-spanish-cased-finetuned-ner', {"use_fast": False}),
        #aggregation_strategy="simple"
        aggregation_strategy="max" # more precision!
        #aggregation_strategy="average"
    )

    chunks = langchain_chunk_text(text)
    all_ner_results = []

    for chunk in chunks:
        ner_results = nlp_ner(chunk)
        all_ner_results.extend(ner_results)

    return all_ner_results